# Recommender systems

## One of the most common uses of big data is to predict and suggest what users may want.  This allows Google to show you relevant ads or to suggest news in Google Now; Amazon to recommend relevant products; Netflix to recommend movies that you might like; or most recently, the famous **Weekly Dicovery** of Spotify.

## All these products are based on systems of recommendation: a information retrieval method to provide users with relevant, yet novel and diverse, information. 

## In this class we will use a pretty famous dataset based on movies ratings, 'MovieLens', to learn the basics of recommender systems. 

## Table of Contents (times are approximated)

1. [Getting and analysing some data (1.5-2 h, typically until break)](#data)
2. [Most popular movies (0.5-1 min)](#popular)
3. [Metrics for recommender systems (2-2.5h)](#metrics)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os

<a id='data'></a>
## 1.1 Load data

We will use MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. This data set consists of:
* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
* Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens [website](https://movielens.org) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip).

Take a look at the readme file!!!

In [5]:
data_root = "./data/ml-100k/"
data_root

'./data/ml-100k/'

In [6]:
!cat {data_root}README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [8]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
datafile = os.path.join(data_root, "u.data")
data = pd.read_csv(datafile, sep='\t', names=columns)
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### A remainder of the numpy library

*Pandas library is nothing alse than numpy under the hood (numpy with steroids, if you like). You can access the data (in matrix from) with he "values" attribute, e.g. data.values*

In [12]:
m = data.values

In [14]:
# The attribute shape provides the shape of the matrix
m.shape

(100000, 4)

In [16]:
# Note that if we return the first column, we get a vector (of 100000 components)
m[:,0].shape

(100000,)

In [20]:
# same with the first row (this time, we get a vector of 4 components)
m[0, :]

array([      196,       242,         3, 881250949])

In [18]:
# access all rows, and first 3 columns 
m[:, 0:3]

array([[ 196,  242,    3],
       [ 186,  302,    3],
       [  22,  377,    1],
       ...,
       [ 276, 1090,    1],
       [  13,  225,    2],
       [  12,  203,    3]])

In [21]:
# access all collumns, and first 3 rows 
m[0:3,:]

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116]])

In [22]:
# access first 10 rows
m[:10, :]
# This is equivalent to data.values[:10]

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       [      244,        51,         2, 880606923],
       [      166,       346,         1, 886397596],
       [      298,       474,         4, 884182806],
       [      115,       265,         2, 881171488],
       [      253,       465,         5, 891628467],
       [      305,       451,         3, 886324817],
       [        6,        86,         3, 883603013]])

In [23]:
# access first column
m[:,0]

array([196, 186,  22, ..., 276,  13,  12])

In [27]:
# Number of users and items
n_users = data.user_id.unique().shape[0]
n_users

943

In [30]:
# Number of items
n_items = data.item_id.unique().shape[0]
n_items

1682

In [31]:
print("There are %s users and %s items" %(n_users, n_items))

There are 943 users and 1682 items


### Brief summary of linear Algebra

In [32]:
m

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [33]:
# dot() calcula el producto escalar
u = np.array([0.1,-0.2])
u.dot(u)

0.05000000000000001

In [35]:
# un producto escalar d eun vector con otro perpendicular da 0
v = np.array([0.2,0.1])
u.dot(v)

0.0

In [36]:
m = np.array([
    [1,2],
    [-1,2],
    [0,3]
])
m

array([[ 1,  2],
       [-1,  2],
       [ 0,  3]])

In [37]:
m.shape, u.shape

((3, 2), (2,))

In [38]:
m.dot(u)

array([-0.3, -0.5, -0.6])

In [40]:
# .T calcula la matriz transpuesta
u.T.dot(m.T)

array([-0.3, -0.5, -0.6])

In [41]:
# la Traza de una matriz es su diagonal. Se saca con diag()
np.diag(m)

array([1, 2])

In [43]:
m[0,0] = 7
m

array([[ 7,  2],
       [-1,  2],
       [ 0,  3]])

In [45]:
ids = np.array([0,2])

In [48]:
cols = np.array([0])

In [50]:
# con un vector puedo coger varias filas en concreto que no estén seguidas
m[ids]

array([[7, 2],
       [0, 3]])

In [51]:
# ojo, he cogido una columna pero me lo devuelve transformado en columna
m[ids, cols]

array([7, 0])

In [52]:
m[[0,2],0]

array([7, 0])

## 1.2 A dictionary for movies and a search tool

In order to analyze the predicted recommendations, let's create a python dictonary that will allow us to translate any item id to the corresponding movie title. Also, let's write a small function that returns the ids of the movies containing some text.

The correspondance between titles and ids is stored in the u.item file

In [53]:
# data_root = "./../data/ml-100k/"

# cargamos el data set que tiene el nombre de los items
items_id_file = os.path.join(data_root, "u.item")
!head $items_id_file

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

### *Simple reminder of dictionaries*

In [55]:
aux = {'hola': 'que haces?', 1: '237'}

In [59]:
# Access value of key='hola'
aux['hola']

'que haces?'

In [60]:
# create new key
aux['adios'] = 'hasta luego'
aux

{'hola': 'que haces?', 1: '237', 'adios': 'hasta luego'}

In [61]:
# Update value of existing key
aux['adios'] = 'hasta nunca'
aux

{'hola': 'que haces?', 1: '237', 'adios': 'hasta nunca'}

In [62]:
# get all keys 
aux.keys()

dict_keys(['hola', 1, 'adios'])

In [63]:
# get all values
list(aux.keys())

['hola', 1, 'adios']

In [64]:
# get all keys and values at the same time
aux.items()

dict_items([('hola', 'que haces?'), (1, '237'), ('adios', 'hasta nunca')])

In [65]:
items_id_file

'./data/ml-100k/u.item'

In [66]:
# Create a dictionary for movie titles and ids
item_dict = {}
with io.open(items_id_file, 'rb') as f: # f es el puntero
    for line in f.readlines():
        record = line.split(b'|') # los string en python vienen en binario, hay que poner una b para leerlos. No es importante.
        item_dict[int(record[0])] = str(record[1])
    
# We can use this dict to see the films a user has seen, for instance. 
for record in data.values[:20]:
    print("User {u} viewed '{m}' and gave a {r} rating".format(
        u=record[0], m=item_dict[record[1]], r=record[2]))    

User 196 viewed 'b'Kolya (1996)'' and gave a 3 rating
User 186 viewed 'b'L.A. Confidential (1997)'' and gave a 3 rating
User 22 viewed 'b'Heavyweights (1994)'' and gave a 1 rating
User 244 viewed 'b'Legends of the Fall (1994)'' and gave a 2 rating
User 166 viewed 'b'Jackie Brown (1997)'' and gave a 1 rating
User 298 viewed 'b'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'' and gave a 4 rating
User 115 viewed 'b'Hunt for Red October, The (1990)'' and gave a 2 rating
User 253 viewed 'b'Jungle Book, The (1994)'' and gave a 5 rating
User 305 viewed 'b'Grease (1978)'' and gave a 3 rating
User 6 viewed 'b'Remains of the Day, The (1993)'' and gave a 3 rating
User 62 viewed 'b'Men in Black (1997)'' and gave a 2 rating
User 286 viewed 'b"Romy and Michele's High School Reunion (1997)"' and gave a 5 rating
User 200 viewed 'b'Star Trek: First Contact (1996)'' and gave a 5 rating
User 210 viewed 'b'To Wong Foo, Thanks for Everything! Julie Newmar (1995)'' and gave a 3 

In [67]:
# Define a function that retrieves all the ids and titles for movies containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in movies titles
    :param ids: dicttionary of {id:title}
    
    :return: a list of (id,title) if text found in titles, and an empty list otherwise.
    """
    # convert input text to lowercase
    text_ = text.lower()
    # find occurances
    search = []
    for id_,title in ids.items():
        if text_ in title.lower():
            search.append((id_, title.lower()))
    
    return search

In [68]:
returnItemId('but', item_dict)

[(240, "b'beavis and butt-head do america (1996)'"),
 (435, "b'butch cassidy and the sundance kid (1969)'"),
 (580,
  "b'englishman who went up a hill, but came down a mountain, the (1995)'"),
 (1401, "b'm. butterfly (1993)'"),
 (1459, "b'madame butterfly (1995)'"),
 (1614, "b'reluctant debutante, the (1958)'"),
 (1621, "b'butterfly kiss (1995)'"),
 (1645, "b'butcher boy, the (1998)'"),
 (1650, "b'butcher boy, the (1998)'")]

## 1.3 Data consistency (always double check everything!!!)

In [81]:
# check whether titles are unique or not. They are not!!!
valores = item_dict.values()
lista = list(valores)
np.unique(np.array(lista)).shape[0]

1664

In [83]:
len(item_dict)

1682

### One work around: create another dict that consolidates ids with the same movie title

Alternatively, you could create a Dataframe from the above dictionary of ids:titles

<pre><code>
dd = pd.DataFrame.from_dict(item_dict, orient='index').reset_index()
dd.columns = ['item_id', 'title']
dd.head()
</code></pre>

find the duplicates (`dd.groupby('title')...`), and create a new id for the items. 

In [ ]:
duplicates_item_dict = {}
# Las claves en "duplicates_item_dict" son los nombres de las películas
# Los valores son una lista de los ids (que pueden ser uno solo, o varios)
for id_, name in list(item_dict.items()):
    # clave: name; valor: list(id_)
    ?

# show the duplicated titles
for k,v in list(duplicates_item_dict.items()):
    if len(v)>1:
        print(k,v)

Create a dict where the key are the original ids, and the values are the unique one. 
We will use this dictionary to remove duplicates in a dataframe.

In [ ]:
unique_id_item_dict ={}
for new_id, old_id_list in enumerate(duplicates_item_dict.values()):
    # key: old_id; value: new_id
    ?

unique_id_item_dict = {old_id: new_id for new_id, old_id_list in enumerate(duplicates_item_dict.values()) 
                                      for old_id in old_id_list}

In [ ]:
unique_id_item_dict

Create another dict mapping moving titles to this new unique id

In [ ]:
unique_item_dict = {unique_id_item_dict[k]:v 
                    for k,v in item_dict.items()}
    
assert(len(set(unique_item_dict.keys())) == 
       len(set(unique_item_dict.values())))

Now we can use our `returnItemId()` mehtod safely =)

In [ ]:
returnItemId('but', unique_item_dict)

In [ ]:
returnItemId('but', item_dict)

## 1.4 Train and test sets

GroupLens provides several splits of the dataset, so that we can check the goodness of our algorithms. See the README file for more  details. Here we will use one of such splits.

Please notice that we have to correct for the non-unique movie's id issue!!

In [84]:
!ls $data_root
# hay varios training y test sets

README	   u.data   u.item	  u1.base  u2.test  u4.base  u5.test  ub.base
allbut.pl  u.genre  u.occupation  u1.test  u3.base  u4.test  ua.base  ub.test
mku.sh	   u.info   u.user	  u2.base  u3.test  u5.base  ua.test


In [85]:
# cogemos uno de ellos, el ua.base
trainfile = os.path.join(data_root, 'ua.base')
!head $trainfile

1	1	5	874965758
1	2	3	876893171
1	3	4	878542960
1	4	3	876893119
1	5	3	889751712
1	6	5	887431973
1	7	4	875071561
1	8	1	875072484
1	9	5	878543541
1	10	3	875693118


In [87]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
trainfile = os.path.join(data_root, "ua.base")
train = pd.read_csv(trainfile, sep='\t', names = columns)
print('There are %s users, %s itmes and %s pairs in the train set' \
      %(train.user_id.unique().shape[0], train.item_id.unique().shape[0], train.item_id.count()))
train.head()


There are 943 users, 1680 itmes and 90570 pairs in the train set


,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [88]:
# same for test
columns = ['user_id', 'item_id', 'rating', 'timestamp']
testfile = os.path.join(data_root, "ua.test")
test = pd.read_csv(testfile, sep='\t', names = columns)
print('There are %s users, %s itmes and %s pairs in the test set' \
      %(test.user_id.unique().shape[0], test.item_id.unique().shape[0], test.item_id.count()))
test.head()


There are 943 users, 1129 itmes and 9430 pairs in the test set


,user_id,item_id,rating,timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201


### Correcting for non-unique movies id 

Use Pandas `apply` method. Note it can be applied to all rows in a column, thus returning a Pandas Series; or it can be applied row-wise, he

*Reminder of lambda functions in Python: is a way of calling short functions (1 line of code), without having to define the function in a separted cell.*

In [ ]:
train['item_id'] = ?

In [ ]:
test['item_id'] = ?

<a id='popular'></a>
## 2. Most popular movies

Recommending popular items is a simple, yet quite effective baseline for recommendation. Indeed, most RS suffer from a strong *popularity bias*, i.e. they tend to recommend popular items more frequently than they should -just because suggesting what is popular is effective!-. There is a lot of research  devote to understand this behaviour and to develop recipies to avoid it. 

Movies can be ranked according to different popularity metrics:
* Most rated movie (it is assumed that this is the most watched movie)
* Most positively rated movie (rating > 4.0)
* Highest rated movie

## 2.1 Most rated movie

In [91]:
# group the train dataset by item and count the number of users using Pandas
mostRated = train.groupby('item_id')['user_id'].count()
mostRated.head()

item_id
1    392
2    121
3     85
4    198
5     79
Name: user_id, dtype: int64

In [92]:
mostRated.head()

item_id
1    392
2    121
3     85
4    198
5     79
Name: user_id, dtype: int64

In [93]:
# sort in descending order
mostRatedSorted = mostRated.sort_values(ascending=False)

In [94]:
mostRatedSorted.head()

item_id
50     495
100    443
181    439
258    412
286    400
Name: user_id, dtype: int64

In [95]:
# add the movie title, using the apply method, returning a Pandas Series in each row
def add_title(row):
    id_ = row[0]
    title_ = item_dict[id_]
    freq_ = row[1]
    new_row = (id_, title_, freq_)
    return pd.Series(new_row, index=['id', 'title', 'freq'])

mostRatedMovies = (mostRatedSorted
                   .reset_index()
                   .apply(add_title, axis=1)
                  )

In [96]:
mostRatedMovies.head()

,id,title,freq
0,50,b'Star Wars (1977)',495
1,100,b'Fargo (1996)',443
2,181,b'Return of the Jedi (1983)',439
3,258,b'Contact (1997)',412
4,286,"b'English Patient, The (1996)'",400


## 2.2 Most positively rated movie

Use the query method to filter values in train, `train.query('rating>4')`, or the traditional form `train[train.rating>4]`.

In [97]:
train.head()

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [102]:
mostPosRated = train[train['rating'] >= 4].groupby('item_id')['user_id'].count()
mostPosRatedSorted = mostPosRated.sort_values(ascending=False)
mostPosRatedSorted.head()

item_id
50     428
100    354
181    331
174    316
98     310
Name: user_id, dtype: int64

In [105]:
mostPosRatedMovies = (mostPosRatedSorted
                   .reset_index()
                   .apply(add_title, axis=1)
                  )

In [106]:
mostPosRatedMovies.head()

,id,title,freq
0,50,b'Star Wars (1977)',428
1,100,b'Fargo (1996)',354
2,181,b'Return of the Jedi (1983)',331
3,174,b'Raiders of the Lost Ark (1981)',316
4,98,"b'Silence of the Lambs, The (1991)'",310


## 2.3 Highest mean rating movie

In [ ]:
# Este ejercicio no me fio de cómo lo he hecho. Mirar los notebooks del profesor

In [113]:
# obtain the highest rated movies, with a minium number of users/ratings.
min_ratings = 50
aux = train.groupby('item_id')['rating'].count()
meanRatedSorted = pd.DataFrame(train.groupby('item_id')['rating'].mean())
meanRatedSorted['mean_rating'] = meanRatedSorted['rating']
meanRatedSorted['count'] = train.groupby('item_id')['user_id'].count()
meanRatedSorted = meanRatedSorted.reset_index()
meanRatedSorted.head()

,item_id,rating,mean_rating,count
0,1,3.859694,3.859694,392
1,2,3.198347,3.198347,121
2,3,3.058824,3.058824,85
3,4,3.545455,3.545455,198
4,5,3.291139,3.291139,79


<div class  = "alert alert-info"> 
** QUESTION **: set the value of *min_ratings* to 1, and re-run the cell. What happens now? Change this value
</div>

<div class  = "alert alert-info"> 
** QUESTION **: Which method is better?? How to measure a recommender system? 
</div>

<div class  = "alert alert-info"> 
** IMPORTANT QUESTION **: When might be useful to recommend popular items?
</div>

<a id='metrics'></a>
## 3. Metrics for recommender systems

As we have seen, even with the simplest solution --aka, recommending popular items-- is difficult to known which technique performs better. For this, there are a number of metrics that allow one to measure the goodness of a recommender system. 

Metrics can be design for measuring the relevance or accuracy of a recommendation, but they can be created for evaluating the novelty of a recommendation, or its diversity. 

For now, we will focus on relevance and accuracy. Several metrics exist:
* Accuracy: rmse, mae.
* Not ranked: Recall@k, Precision@k.
* With rank disccount: map@k, ndcg@k.
* With rank ordering: mean percentile rank.

We will be definiing some of them whitin this class. For the moment, let's talk about precision and recall.

## 3.1 Precision and recall

<img src="https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg" alt="Precision and Recall in IR" style="float: right; width: 300px"/>

The concept of precision and recall comes form the world of information retrieval, have a look at the wikipedia:

https://en.wikipedia.org/wiki/Precision_and_recall

From this entry:

 * "**precision** (also called positive predictive value) is the fraction of retrieved instances that are relevant".
 * "**recall** (also known as sensitivity) is the fraction of relevant instances that are retrieved".

<br />
<div class  = "alert alert-info"> 
** QUESTION **: how do we know if some movie, unknown to the user, is relevant?
</div>

In other words, we cannot measure a false positive --something recommended that was not relevant--. In this regard, only recall-oriented metrics have an actual meaning in RS. Nonetheless, its common practice to define both metrics in RS as follows:
 
### $$\mathrm{recall}@N = \frac{\sum_{k=1}^N rel(k)}{\sum_{i\in \mathcal{I}_u} 1}$$
### $$\mathrm{precision}@N = \frac{\sum_{k=1}^N rel(k)}{N}$$

Here, $\mathcal{I}_u$ is the set of items adopted by user $u$, and $rel(k)$ is the relevance of a recommendation at position k in the list of recommendations. For ratings, the relevance could be defined as those movies rated above a certain threshold, e.g. $r_{ui}>4.0$. 

**Important to note: since precision is pretty much the same as recall in RS, metrcis usch as the *area under the ROC curve* doesn't have any meaning!!**

<div class = "alert alert-success">
As an example, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />
**What would be the recall and precision @1? and @2?**
<br />
**What do you think of recommending Batman? Is a bad or a good recommendation?**
</div>

Please notice that there isn't any actual difference between precision and recall in the context of RS: both measure the relevance of the recommendations, and tell nothing about items recommended that haven't been adopted by the user. Thus, it make sense to define a normalized recall as:

### $$\mathrm{recall}@N = \frac{\sum_{i=1}^N rel_i}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1})$$

This way, results are normalized to 1 always.

<div class="alert alert-success">
**Exercise** Implement the above definition of recall
</div>

In [117]:
def recall_at_n(N, test, recommended, train=None):
    """
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return the recall
    """
    if train is not None: # Remove items in train
        rec_true = []
        for r in recommended:
            ?
    else:
        rec_true = recommended    
    intersection = len(set(recommended[:N]) & set(test)) # con el set calculo la interseccion, porque set me elimina elementos duplicados
    return intersection / float(np.minimum(N, len(test)))

In [118]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Batman (1989)', 'Madame Butterfly (1995)']

In [119]:
recall_at_n(1, seen, recommended)

0.0

In [120]:
recall_at_n(2, seen, recommended)

0.5

In [121]:
# Check it's well normalized
print(recall_at_n(3, seen, recommended))
print(recall_at_n(10, seen, recommended))
print(recall_at_n(100, seen, recommended))

0.3333333333333333
0.25
0.25


### Now, use this implementation to measure the efficiency of the popularity baselines in the test set. Use the top-5 movies, for instance

In [ ]:
mostRatedMovies[:5,1:]

In [ ]:
positiveRatedMovies[:5,1:]

In [ ]:
meanRateMovies[:5,1:]

In [ ]:
train.head()

*Since `recall_at_n` takes both train and test list per user, we need to create a dataset with the list of movies seen in train and test*

Thus, get the list of movies per user in train and test, and join the two dataframes. For the join, use the pandas method `merge`.

In [ ]:
# get movies in train per user. For this, group by user and get a list of item ids.
trainUsersGrouped = ?


In [ ]:
# same with test data
testUsersGrouped = ?

In [ ]:
# make the join: use pandas merge method
joined = ?

In [ ]:
joined.head()

In [ ]:
joined.item_id_test.head()

In [ ]:
# How would you access values in test?
?

In [ ]:
# This second method is easier if we want to access several columns at once, and operate over them.
# For instance, if we like to concatenate both train and test list, we will do:
?

In [ ]:
# Use the above method to calculate the recall of the mostRatedMovies recommendation, for each user:
?

*As you can see, some users have a quite large recall (0.5), while for others is small (e.g, 0.14). Let's calculate the mean.*

In [ ]:
topN = 30
# calculate the average recall across all users for mostRatedMovies recommendation
recall_per_user = ?
recall_per_user.mean()

In [ ]:
# calculate the average recall across all users for positiveRatedMovies recommendation
?

In [ ]:
# calculate the average recall across all users for meanRatedMovies recommendation
?

## 3.2 Mean Averaged Precision (MAP) -- Advanced material

Previous metrics did not account for the ranking of the recommendation, i.e. the relative position of a movie within the sorted list of recommendations. **But orders matters!** Metrics like MAP, MRR or NDCG try to tackle down this problem. 

From the blog *http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/*:

> Here’s another way to understand average precision. Wikipedia says AP is used to score document retrieval. You can think of it this way: you type something in Google and it shows you 10 results. It’s probably best if all of them were relevant. If only some are relevant, say five of them, then it’s much better if the relevant ones are shown first. It would be bad if first five were irrelevant and good ones only started from sixth, wouldn’t it? AP score reflects this.

Implementation taken from:

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py



## Average Precision 

The Average Precision is definied as:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N P(k) \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$

where $P(k)$ is the precision at cut-off in the item list, i.e. the ratio of the number of recommended items adopted, up to the position k, over the number k. Thus:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N \left(\sum_{i=1}^k rel(i)\right)/k \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$



<div class = "alert alert-success">
Following the example above, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />

<div class = "alert alert-success">
**Calculate AP@1**
<br /><br />
First, *rel(1)=0*, because Batman was not viewed. Also, *P(1) = 0*. Thus, AP@1=0.
<br />
**Calculate AP@2**
<br /><br />
As before, *rel(1)=0*, so the first term does not contribute. For the second term, *rel(2)=1*, so that *P(2)=0.5*. The numerator is hence:
<br /><br />
$P(1)*rel(1)+P(2)*rel(2)=0*0+0.5*1$
<br /><br />
For the denominator, $N=2$ and $\sum_{i\in \mathcal{I}_u} 1)=4$, thus:
<br /><br />
AP@2 = 0.5/2 = 0.25
</div>

Let's now implement it =)

In [ ]:
def apk(N, test, recommended, train=None):
    """
    Computes the average precision at N given recommendations.
    
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return The average precision at N over the test set
    """
    if train is not None: 
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    predicted = rec_true[:N] # top-k predictions
    
    score = 0.0 # This will store the numerator
    num_hits = 0.0 # This will store the sum of rel(i)

    for i,p in enumerate(predicted):
        if p in test and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits/(i+1.0)

    return score / min(len(test), N)

In [ ]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Madame Butterfly (1995)', 'Batman (1989)']

In [ ]:
apk(1, seen, recommended)

In [ ]:
apk(2, seen, recommended)

In [ ]:
apk(3, seen, recommended)

## MAP

Mean avergae precision is nothing else than the AP averaged across users ;)

Apply it to popularity baselines

In [ ]:
?

<div class="alert alert-success">
The rest of the class is covered in a different notebook
</div>